In [1]:
import pandas
import datetime
import numpy
from sklearn import linear_model
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import ListedColormap
from matplotlib.colors import Normalize
from matplotlib import ticker
from matplotlib import rcParams

In [2]:
# setup some cutoff values we'll use in the analysis
velCutoffUpper = 2000.
velCutoffLower = 0.
numPointsCutoffMLTMLAT = 250
mlatCutOffUpper = 70.
probOccCutoff = 0.2

In [ ]:
velGmagDF = pandas.read_csv("../data/processed-vels-geomag.txt", sep=' ')
velGmagDF = velGmagDF.drop('Unnamed: 0', axis=1)
# Discard unwanted values
# We'll only consider those velocities 
# which lie between 0 and 2500 m/s
# and located below 70 MLAT
velGmagDF = velGmagDF[ (velGmagDF["vSaps"] > velCutoffLower) \
                        & (velGmagDF["vSaps"] < velCutoffUpper)\
                       ].reset_index(drop=True)
velGmagDF = velGmagDF[ velGmagDF["MLAT"] < mlatCutOffUpper ].reset_index(drop=True)
# Now filter out velocities which have very few rate of occ.
# We calculat the prob and remove every measurement below 0.2 prob of occ.
mlatMLTDstCountDF = velGmagDF.groupby( ["MLAT", "normMLT", "dst_bin"] )["vSaps"].count().reset_index()
mlatMLTDstCountDF.columns = [ "MLAT", "normMLT", "dst_bin", "count" ]
dstMaxCntDF = mlatMLTDstCountDF.groupby( ["dst_bin"] )["count"].max().reset_index()
dstMaxCntDF.columns = [ "dst_bin", "maxCntDst" ]
mlatMLTDstCountDF = pandas.merge( mlatMLTDstCountDF, dstMaxCntDF, on=[ "dst_bin" ] )
mlatMLTDstCountDF["probOcc"] = mlatMLTDstCountDF["count"]/mlatMLTDstCountDF["maxCntDst"]
mlatMLTDstCountDF = mlatMLTDstCountDF[ mlatMLTDstCountDF["probOcc"] > probOccCutoff ].reset_index(drop=True)
# Filter out MLATs and MLTs (at the Dst bins)
# where number of measurements is low. We do
# this by merging the mlatMLTDstCountDF with velDF.
velGmagDF = pandas.merge( velGmagDF,\
                         mlatMLTDstCountDF,\
                         on=[ "MLAT", "normMLT", "dst_bin" ] )
velGmagDF = velGmagDF[ [ "normMLT", "MLAT", "vSaps",\
                        "azim", "dst_bin", "dst_index", "count", "maxCntDst" ] ]
# Divide the velocities into bins
velBins = [ v for v in range(0,int(velCutoffUpper)+100,100) ]
velGmagDF = pandas.concat( [ velGmagDF, \
                    pandas.cut( velGmagDF["vSaps"], \
                               bins=velBins ) ], axis=1 )
velGmagDF.columns = [ "normMLT", "MLAT", "vSaps",\
                        "azim", "dst_bin", "dst_index", "count",\
                         "maxCntDst", "vel_bin" ]
# velGmagDF.head()
# Get a DF with mean Dst in each bin
dstMeanDF = velGmagDF.groupby( ["dst_bin"] ).mean()["dst_index"].astype(int).reset_index()
dstMeanDF.columns = [ "dst_bin", "dst_mean" ]
velGmagDF = pandas.merge( velGmagDF, dstMeanDF, on=["dst_bin"] )
velGmagDF = velGmagDF.sort( ["dst_mean"], ascending=False ).reset_index(drop=True)
maxSapsVel = numpy.ceil( velGmagDF["vSaps"].max()/1000 ) * 1000.
velGmagDF["norm_vels"] = velGmagDF["vSaps"]/maxSapsVel
# velGmagDF.head()
velGmagDF["dst_bin"].unique()

array(['(-10, 10]', '(-25, -10]', '(-50, -25]', '(-75, -50]', '(-150, -75]'], dtype=object)

In [ ]:
# Now fir a skewed normal distribution for the velocities
# at every MLT, MLAT and dst_bin! Store the results in a DF
fitShapeArr = []
fitLocArr = []
fitScaleArr = []
normMLTArr = []
mlatArr = []
dstBinArr = []
dstMeanArr = []
uniqMLTs = velGmagDF["normMLT"].unique().tolist()
uniqMLATs = velGmagDF["MLAT"].unique().tolist()
uniqDstBins = velGmagDF["dst_bin"].unique().tolist()
for currMlt in uniqMLTs:
    for currMlat in uniqMLATs:
        for currDstBin in uniqDstBins:
            subDF = velGmagDF[ (velGmagDF["normMLT"] == currMlt) & \
                  (velGmagDF["dst_bin"] == currDstBin) & \
                  (velGmagDF["MLAT"] == currMlat) ]
            # If we have values in this DF fit a skewnorm distribution
            if subDF.shape[0] > 0:
                shape, loc, scale = stats.skewnorm.fit(subDF["norm_vels"], loc=0, scale=1)
                fitShapeArr.append( shape )
                fitLocArr.append( loc )
                fitScaleArr.append( scale )
                normMLTArr.append( currMlt )
                mlatArr.append( currMlat )
                dstBinArr.append( currDstBin )
                dstMeanArr.append( velGmagDF[velGmagDF["dst_bin"] == currDstBin]["dst_mean"].tolist()[0] )
# convert the data to a dataframe
fitDF = pandas.DataFrame(
    {'shape': fitShapeArr,
     'loc': fitLocArr,
     'scale': fitScaleArr,
     'norm_mlt': normMLTArr,
     'mlat': mlatArr,
     'dst_bin': dstBinArr,
     'dst_mean': dstMeanArr,
    })
fitDF.head()
# There are some extreme values in fitDF, take them out
# Some values of shape go beyond 10^8
fitDF = fitDF[ fitDF["shape"] <= 500. ]
# Save the fitting results to a DF so that we can use it later
fitDF.to_csv( "../data/fit-skewed-gaussian.txt",\
                      sep=' ', index=False )

In [ ]:
# Make a few plots of the fitting to get an estimate of how things are!
# Our velocities range from 0 to 2000 m/s
velsArr = numpy.arange(0.,2100.,100.)#/2000.
selMLAT = 59.5#59.#
selNormMLT = -3.#-6.#
selFitDF = fitDF[ (fitDF["norm_mlt"] == selNormMLT) &\
                (fitDF["mlat"] == selMLAT) ]
selVelDF = velGmagDF[ (velGmagDF["normMLT"] == selNormMLT) & \
                  (velGmagDF["MLAT"] == selMLAT) ]
# print selFitDF["dst_bin"].unique()

In [ ]:
f = plt.figure(figsize=(12, 8))
axArr = []
axArr.append( f.add_subplot(2,2,1) )
axArr.append( f.add_subplot(2,2,2) )
axArr.append( f.add_subplot(2,2,3) ) 
axArr.append( f.add_subplot(2,2,4) )
# axArr.append( f.add_subplot(3,2,5) )
for ind, currDstBin in enumerate( selFitDF["dst_bin"].unique() ):
    shape = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["shape"].tolist()[0]
    loc = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["loc"].tolist()[0]
    scale = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["scale"].tolist()[0]
    subVelDF = selVelDF[ selVelDF["dst_bin"] == currDstBin ]
    velScale = subVelDF["vSaps"].value_counts(bins=velsArr[::2]).max()
    velKernel = stats.gaussian_kde( subVelDF["vSaps"], bw_method="scott" )
    kernel_adjusted = (velKernel.pdf( velsArr )/max(velKernel.pdf( velsArr ))) * velScale
    pdf_fitted = stats.skewnorm.pdf(velsArr/2000., shape, loc, scale)
    fitVals = pdf_fitted/max(pdf_fitted) * velScale
    axArr[ind].plot(velsArr, kernel_adjusted, 'r-', lw=5, alpha=0.6)
    axArr[ind].plot(velsArr, fitVals, 'b--', lw=5, alpha=0.6)
    axArr[ind].set_xlabel("Velocities [m/s]")
    axArr[ind].set_ylabel("counts")
    printText = "Lat : " + str(selMLAT) +\
                "\n" + "MLT : " + str(selNormMLT) +\
                "\n" + "Dst : " + currDstBin
    axArr[ind].text(0.7, 0.7,  printText,
            transform=axArr[ind].transAxes,
            color='green', fontsize=10)
    subVelDF["vSaps"].hist(ax=axArr[ind])
plt.savefig("../figs/skewed-gaussian-fit-test.pdf",bbox_inches='tight')

In [ ]:
# Seaborn styling
sns.set_style("darkgrid")
sns.set_context("paper")
seaMap = ListedColormap(sns.color_palette("Reds"))
# Plot using matplotlib
fig1 = plt.figure()
ax = fig1.add_subplot(111)
fitDF[ fitDF["dst_bin"] == "(-50, -25]" ].plot(x="norm_mlt", y="shape", kind="scatter", ax=ax)

In [ ]:
selParam = "shape"
selDF = fitDF#[ (fitDF["dst_bin"] == "(-150, -75]") ] # & (fitDF["norm_mlt"] == -6.)
selDF = selDF[ selDF["shape"] < 2000.]
sns.swarmplot( x="mlat", y=selParam, hue="dst_mean", data=selDF[ ["mlat", "norm_mlt", "dst_bin", "dst_mean" , selParam] ])

In [ ]:
f = plt.figure(figsize=(12, 8))
ax = f.add_subplot(1,1,1)

sns.set_style("darkgrid")
sns.set_context("paper")
seaMap = ListedColormap(sns.color_palette("Reds"))

fitDF[fitDF["shape"] < 200.].plot( kind='scatter',
              x='dst_mean',
              y='norm_mlt',
              c='scale',
              s=50, cmap=seaMap,ax=ax)
# ax.set_xlabel("MLT", fontsize=14)
# ax.set_ylabel("MLAT", fontsize=14)

In [ ]:
# Fit a multivariate linear model to get estimates of scale, shape and loc
model = linear_model.LinearRegression()
scaleFit = linear_model.LinearRegression().fit(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ], fitDF["scale"])
shapeFit = linear_model.LinearRegression().fit(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ], fitDF["shape"])
locFit = linear_model.LinearRegression().fit(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ], fitDF["loc"])

In [ ]:
print "scale fitting--->", scaleFit.intercept_, scaleFit.coef_
print "shape fitting--->", shapeFit.intercept_, shapeFit.coef_
print "loc fitting--->", locFit.intercept_, locFit.coef_

In [ ]:
fitDF["pred_scale"] = scaleFit.predict(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ])
fitDF["pred_shape"] = shapeFit.predict(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ])
fitDF["pred_loc"] = locFit.predict(fitDF[ ["dst_mean", "mlat", "norm_mlt"] ])

In [ ]:
f = plt.figure(figsize=(12, 8))
ax1 = f.add_subplot(2,1,1)
ax2 = f.add_subplot(2,1,2)

selDF = fitDF[ (fitDF["dst_bin"] == "(-150, -75]") ] # & (fitDF["norm_mlt"] == -6.)
selDF["pred_shape"] = shapeFit.predict(selDF[ ["dst_mean", "mlat", "norm_mlt"] ])
selDF.plot( kind='scatter',
              x='mlat',
              y='norm_mlt',
              c='shape',
              s=50, cmap=seaMap,ax=ax1)
selDF.plot( kind='scatter',
              x='mlat',
              y='norm_mlt',
              c='pred_shape',
              s=50, cmap=seaMap,ax=ax2)

In [ ]:
# Make a few plots of the fitting to get an estimate of how things are!
# Our velocities range from 0 to 2000 m/s
velsArr = numpy.arange(0.,2100.,100.)
selMLAT = 59.#59.#
selNormMLT = -3.#-6.#
selFitDF = fitDF[ (fitDF["norm_mlt"] == selNormMLT) &\
                (fitDF["mlat"] == selMLAT) ]
selVelDF = velGmagDF[ (velGmagDF["normMLT"] == selNormMLT) & \
                  (velGmagDF["MLAT"] == selMLAT) ]

f = plt.figure(figsize=(12, 8))
axArr = []
axArr.append( f.add_subplot(3,2,1) )
axArr.append( f.add_subplot(3,2,2) )
axArr.append( f.add_subplot(3,2,3) ) 
axArr.append( f.add_subplot(3,2,4) )
axArr.append( f.add_subplot(3,2,5) )
for ind, currDstBin in enumerate( selFitDF["dst_bin"].unique() ):
    shape = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["pred_shape"].tolist()[0]
    loc = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["pred_loc"].tolist()[0]
    scale = selFitDF[ selFitDF["dst_bin"] == currDstBin ]["pred_scale"].tolist()[0]
    subVelDF = selVelDF[ selVelDF["dst_bin"] == currDstBin ]
    velScale = subVelDF["vSaps"].value_counts(bins=velsArr[::2]).max()
    pdf_fitted = stats.skewnorm.pdf(velsArr, shape, loc, scale)
    fitVals = pdf_fitted/max(pdf_fitted) * velScale
    axArr[ind].plot(velsArr, fitVals, 'r-', lw=5, alpha=0.6)
    axArr[ind].set_xlabel("Velocities [m/s]")
    axArr[ind].set_ylabel("counts")
    printText = "Lat : " + str(selMLAT) +\
                "\n" + "MLT : " + str(selNormMLT) +\
                "\n" + "Dst : " + currDstBin
    axArr[ind].text(0.7, 0.7,  printText,
            transform=axArr[ind].transAxes,
            color='green', fontsize=10)
    subVelDF["vSaps"].hist(ax=axArr[ind])
# plt.savefig("../figs/skewed-gaussian-fit-test.pdf",bbox_inches='tight')